In [1]:
from models import EfficientNet3D
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
num_classes = 3
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': num_classes}, in_channels=1)

In [3]:
model = model.to(device)
inputs = torch.randn((1, 45, 15, 15, 15)).to(device)
labels = torch.tensor([0]).to(device)
# test forward

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticSamePadding(
        32, 16, kernel_siz

In [6]:
previous_prediction_1 = torch.randn((1, 3))
previous_prediction_2 = torch.randn((1, 3))
# Concatenate the previous predictions together along dimension 2
previous_predictions = torch.cat((previous_prediction_1, previous_prediction_2), dim=1).cuda()
print("Previous prediction size outside", previous_predictions.shape)

Previous prediction size outside torch.Size([1, 6])


In [7]:
for epoch in range(1):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs, previous_predictions)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print(outputs.shape)
print(outputs.cpu().detach().numpy())

[1] loss: 0.910
Finished Training
torch.Size([1, 3])
[[ 0.26392272 -0.03697111 -0.0320048 ]]
